In [1]:
!pip install torch_geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00a 0:00:01


In [2]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git


Cloning into 'hackaton'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 81 (delta 1), reused 1 (delta 1), pack-reused 77 (from 1)
Receiving objects: 100% (81/81), 105.83 MiB | 43.84 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [3]:
%cd hackaton/

/kaggle/working/hackaton


In [4]:
!gdown --folder https://drive.google.com/drive/folders/1Gf0mFRaqg_-cMAOUCkZvIemTcEpzh4XF?usp=sharing -O datasets


Retrieving folder contents
Retrieving folder 1pjSkW1F77JobB9WM0D6upyV4MVAh-Bhh A
Processing file 1V6fMoOAz1PmmX2J2sC6odnX8aHvsLkXI test.json.gz
Processing file 1Mr9pkFhefAY-kvByNDQO1hkPFCrWrbVT train.json.gz
Retrieving folder 1MWCcdJHVxpsDkduapri9NCB7SEtj8bcS B
Processing file 1--jQFe8IrzBy9POUAVdyjjjYeJoIv-ww test.json.gz
Processing file 1QGpZ3tEN693onLtNKpVCfgNuurjhxiNK train.json.gz
Retrieving folder 1DJ0itapAg68xT4ohngXeLSN4lL9o6aPV C
Processing file 1vhelRWElU7h4LMdzqP7Ky85QKKQ188MD test.json.gz
Processing file 1Xz59-7TJ2a-hbSB40ZVg86pR0_7njEgY train.json.gz
Retrieving folder 1pFJqHvELSrcmF9M0Op3qncGSohZq6lmU D
Processing file 1xS_NWAMf-6BSLzzZwUWbnClJqLXll6iL test.json.gz
Processing file 1bTRtEZvLQsqTRotd1V_tUtBB6j6wsIfl train.json.gz
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1V6fMoOAz1PmmX2J2sC6odnX8aHvsLkXI
From (redirected): https://drive.google.com/uc?

In [5]:
!ls -lh datasets

total 16K
drwxr-xr-x 2 root root 4.0K May 27 16:06 A
drwxr-xr-x 2 root root 4.0K May 27 16:06 B
drwxr-xr-x 2 root root 4.0K May 27 16:06 C
drwxr-xr-x 2 root root 4.0K May 27 16:06 D


In [24]:
def add_zeros(data):
        if data.x is None:
            # Change dtype to torch.float
            data.x = torch.ones((data.num_nodes, 8), dtype=torch.float)
        return data

In [7]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed()


import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm

In [25]:
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool
from torch_geometric.utils import degree
import math

class MLPLayer(nn.Module):
    """Multi-layer perceptron layer with batch normalization and dropout"""
    def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.1):
        super(MLPLayer, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.bn(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

class GINPlusLayer(nn.Module):
    """Enhanced GIN layer with edge features and improved aggregation"""
    def __init__(self, in_dim, hidden_dim, edge_dim, dropout=0.1):
        super(GINPlusLayer, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.edge_dim = edge_dim
        
        # Edge feature transformation
        self.edge_encoder = nn.Sequential(
            nn.Linear(edge_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # Node feature transformation
        self.node_encoder = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # MLP for aggregation (as per GIN)
        self.mlp = MLPLayer(hidden_dim, hidden_dim * 2, hidden_dim, dropout)
        
        # Learnable epsilon parameter
        self.eps = nn.Parameter(torch.zeros(1))
        
        # Attention mechanism for edge-aware aggregation
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1),
            nn.LeakyReLU(0.2)
        )
        
    def forward(self, x, edge_index, edge_attr, virtual_node=None):
        # Transform node features
        x_transformed = self.node_encoder(x)
        
        # Transform edge features
        edge_feat = self.edge_encoder(edge_attr)
        
        # Message passing with edge-aware attention
        row, col = edge_index
        
        # Compute attention weights
        node_i = x_transformed[row]  # Source nodes
        node_j = x_transformed[col]  # Target nodes
        
        # Combine node and edge features for attention
        attention_input = torch.cat([node_i + edge_feat, node_j], dim=-1)
        attention_weights = self.attention(attention_input)
        attention_weights = F.softmax(attention_weights, dim=0)
        
        # Weighted message aggregation
        messages = (node_j + edge_feat) * attention_weights
        
        # Aggregate messages for each node
        out = torch.zeros_like(x_transformed)
        out.index_add_(0, row, messages)
        
        # Add virtual node contribution if provided
        if virtual_node is not None:
            out = out + virtual_node.expand_as(out)
        
        # Apply GIN update rule: (1 + eps) * x + aggregated_messages
        out = (1 + self.eps) * x_transformed + out
        
        # Apply MLP
        out = self.mlp(out)
        
        return out

class VirtualNode(nn.Module):
    """Virtual node implementation for global graph representation"""
    def __init__(self, hidden_dim, dropout=0.1):
        super(VirtualNode, self).__init__()
        self.hidden_dim = hidden_dim
        
        # Virtual node embedding
        self.virtual_emb = nn.Parameter(torch.randn(1, hidden_dim))
        
        # MLPs for virtual node updates
        self.vn_mlp = MLPLayer(hidden_dim, hidden_dim * 2, hidden_dim, dropout)
        self.node_mlp = MLPLayer(hidden_dim, hidden_dim * 2, hidden_dim, dropout)
        
        # Attention for virtual node interaction
        self.vn_attention = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, x, batch):
        batch_size = batch.max().item() + 1
        
        # Aggregate node features per graph for virtual node update
        vn_input = global_mean_pool(x, batch)  # [batch_size, hidden_dim]
        
        # Update virtual node representation
        vn_feat = self.virtual_emb.expand(batch_size, -1)
        vn_updated = self.vn_mlp(vn_feat + vn_input)
        
        # Compute attention between virtual node and regular nodes
        vn_expanded = vn_updated[batch]  # Expand to match node dimensions
        attention_input = torch.cat([x, vn_expanded], dim=-1)
        attention_weights = torch.softmax(self.vn_attention(attention_input), dim=0)
        
        # Apply attention to virtual node contribution
        vn_contribution = vn_expanded * attention_weights
        
        return vn_contribution, vn_updated

class NoiseFilter(nn.Module):
    """Noise filtering module for handling noisy labels"""
    def __init__(self, hidden_dim, num_classes, confidence_threshold=0.8):
        super(NoiseFilter, self).__init__()
        self.confidence_threshold = confidence_threshold
        self.num_classes = num_classes
        
        # Confidence estimation network
        self.confidence_net = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
        # Label correction network
        self.correction_net = nn.Sequential(
            nn.Linear(hidden_dim + num_classes, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, num_classes)
        )
        
    def forward(self, graph_emb, predictions, labels=None):
        # Estimate confidence in predictions
        confidence = self.confidence_net(graph_emb)
        
        if labels is not None and self.training:
            # During training, identify potentially noisy samples
            pred_probs = F.softmax(predictions, dim=-1)
            max_prob, pred_labels = torch.max(pred_probs, dim=-1)
            
            # Samples with low confidence or disagreement with labels are potentially noisy
            label_agreement = (pred_labels == labels).float().unsqueeze(-1)
            noise_score = 1.0 - (confidence * label_agreement)
            
            # For high noise score samples, attempt label correction
            correction_input = torch.cat([graph_emb, F.one_hot(labels, self.num_classes).float()], dim=-1)
            corrected_logits = self.correction_net(correction_input)
            
            # Blend original and corrected predictions based on noise score
            final_logits = (1 - noise_score) * predictions + noise_score * corrected_logits
            
            return final_logits, confidence, noise_score
        else:
            return predictions, confidence, None

class GINPlusModel(nn.Module):
    """Complete GIN+ model with virtual node and noise filtering"""
    def __init__(self, in_dim=8, edge_dim=8, hidden_dim=300, out_classes=6, 
                 num_layers=5, dropout=0.1, use_virtual_node=True, 
                 confidence_threshold=0.8):
        super(GINPlusModel, self).__init__()
        
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.use_virtual_node = use_virtual_node
        self.dropout = dropout
        
        # Input feature transformation
        self.input_encoder = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # GIN+ layers
        self.gin_layers = nn.ModuleList([
            GINPlusLayer(hidden_dim, hidden_dim, edge_dim, dropout)
            for _ in range(num_layers)
        ])
        
        # Virtual node
        if use_virtual_node:
            self.virtual_node = VirtualNode(hidden_dim, dropout)
        
        # Graph-level pooling
        self.pool = [
            global_add_pool,
            global_mean_pool,
            global_max_pool
        ]
        
        # Final classifier
        pooled_dim = hidden_dim * 3  # Concatenation of add, mean, max pooling
        self.classifier = nn.Sequential(
            nn.Linear(pooled_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, out_classes)
        )
        
        # Noise filtering
        self.noise_filter = NoiseFilter(pooled_dim, out_classes, confidence_threshold)
        
        # Initialize parameters
        self._init_parameters()
        
    def _init_parameters(self):
        """Initialize model parameters"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
    
    def forward(self, x, edge_index, edge_attr, batch, labels=None):
        # Input encoding
        x = self.input_encoder(x)
        
        # GIN+ layers with virtual node
        vn_contribution = None
        for i, gin_layer in enumerate(self.gin_layers):
            if self.use_virtual_node:
                if i == 0:
                    # Initialize virtual node contribution
                    vn_contribution, _ = self.virtual_node(x, batch)
                else:
                    # Update virtual node
                    vn_contribution, _ = self.virtual_node(x, batch)
            
            x = gin_layer(x, edge_index, edge_attr, vn_contribution)
            
            # Apply residual connection and layer normalization
            if i > 0:
                x = x + x_prev
            x_prev = x
        
        # Graph-level representation
        graph_emb_list = []
        for pool_fn in self.pool: # Iterate through the list of functions
            graph_emb_list.append(pool_fn(x, batch))

        graph_emb = torch.cat(graph_emb_list, dim=-1)
        
        # Classification
        logits = self.classifier(graph_emb)
        
        # Noise filtering
        filtered_logits, confidence, noise_score = self.noise_filter(graph_emb, logits, labels)
        
        return {
            'logits': filtered_logits,
            'confidence': confidence,
            'noise_score': noise_score,
            'graph_embedding': graph_emb
        }
    
    def get_embeddings(self, x, edge_index, edge_attr, batch):
        """Get graph embeddings without classification"""
        with torch.no_grad():
            output = self.forward(x, edge_index, edge_attr, batch)
            return output['graph_embedding']

In [26]:
cross_entropy_val = nn.CrossEntropyLoss

mean = 1e-8
std  = 1e-9
encoder_features =512
total_epochs = 5

class ncodLoss(nn.Module):
    def __init__(self, labels, n=50000, C=100, ratio_consistency=0, ratio_balance=0):
        super(ncodLoss, self).__init__()

        self.C = C
        self.USE_CUDA = torch.cuda.is_available()
        self.n = n

        self.ratio_consistency = ratio_consistency
        self.ratio_balance = ratio_balance


        self.u = nn.Parameter(torch.empty(n, 1, dtype=torch.float32))
        self.init_param(mean=mean,std=std)

        self.beginning = True
        self.prev_phi_x_i = torch.rand((n, encoder_features))
        self.phi_c = torch.rand((C, encoder_features))
        self.labels = labels
        self.bins = []

        for i in range(0, C):
            self.bins.append(np.where(self.labels == i)[0])


    def init_param(self, mean= 1e-8, std= 1e-9):
        torch.nn.init.normal_(self.u, mean=mean, std=std)


    def forward(self, index, f_x_i, y, phi_x_i, flag, epoch):

        if len(f_x_i) > len(index):
            f_x_i_1, f_x_i_2 = torch.chunk(f_x_i, 2)
            phi_x_i_1, phi_x_i_2 = torch.chunk(phi_x_i, 2)
        else:
            f_x_i_1 = f_x_i
            phi_x_i_1 = phi_x_i

        eps = 1e-4

        u = self.u.to(index.device)[index]



        if (flag == 0):
            if self.beginning:
                percent = math.ceil((50 - (50 / total_epochs) * epoch) + 50)
                for i in range(0, len(self.bins)):
                    class_u = self.u.detach()[self.bins[i]]
                    bottomK = int((len(class_u) / 100) * percent)
                    important_indexs = torch.topk(class_u, bottomK, largest=False, dim=0)[1]
                    self.phi_c[i] = torch.mean(self.prev_phi_x_i[self.bins[i]][important_indexs.view(-1)],
                                                      dim=0)

            phi_c_norm = self.phi_c.norm(p=2, dim=1, keepdim=True)
            h_c_bar = self.phi_c.div(phi_c_norm)
            self.h_c_bar_T = torch.transpose(h_c_bar, 0, 1)
            self.beginning = True

        self.prev_phi_x_i[index] = phi_x_i_1.detach()

        f_x_softmax = F.softmax(f_x_i_1, dim=1)

        phi_x_i_1_norm = phi_x_i_1.detach().norm(p=2, dim=1, keepdim=True)
        h_i = phi_x_i_1.detach().div(phi_x_i_1_norm)

        y_bar = torch.mm(h_i, self.h_c_bar_T)
        y_bar = y_bar * y
        y_bar_max = (y_bar > 0.000).type(torch.float32)
        y_bar = y_bar * y_bar_max

        u = u * y

        f_x_softmax = torch.clamp((f_x_softmax + u.detach()), min=eps, max=1.0)
        L1 = torch.mean(-torch.sum((y_bar) * torch.log(f_x_softmax), dim=1))

        y_hat = self.soft_to_hard(f_x_i_1.detach())

        L2 = F.MSE_loss((y_hat + u), y, reduction='sum') / len(y)
        L1 += L2



        if self.ratio_balance > 0:
            avg_prediction = torch.mean(f_x_softmax, dim=0)
            prior_distr = 1.0 / self.C * torch.ones_like(avg_prediction)

            avg_prediction = torch.clamp(avg_prediction, min=eps, max=1.0)

            balance_kl = torch.mean(-(prior_distr * torch.log(avg_prediction)).sum(dim=0))

            L1 += self.ratio_balance * balance_kl

        if (len(f_x_i) > len(index)) and (self.ratio_consistency > 0):
            consistency_loss = self.consistency_loss( f_x_i_1, f_x_i_2)

            L1 += self.ratio_consistency * torch.mean(consistency_loss)


        return L1


    def consistency_loss(self,  f_x_i_1, f_x_i_2):
        preds1 = F.softmax( f_x_i_1, dim=1).detach()
        preds2 = F.log_softmax(f_x_i_2, dim=1)
        loss_kldiv = F.kl_div(preds2, preds1, reduction='none')
        loss_kldiv = torch.sum(loss_kldiv, dim=1)
        return loss_kldiv

    def soft_to_hard(self, x):
        with torch.no_grad():
            return (torch.zeros(len(x), self.C)).cuda().scatter_(1, (x.argmax(dim=1)).view(-1, 1), 1)

In [27]:
# Wraps ncodLoss to accept model output dict
class NcodLossWrapper(nn.Module):
    def __init__(self, ncod_loss):
        super().__init__()
        self.ncod_loss = ncod_loss

    def forward(self, outputs, labels, index=None, flag=0, epoch=0):
        logits = outputs['logits']
        phi_x_i = outputs['graph_embedding']
        C = logits.size(1)
        y_onehot = F.one_hot(labels, num_classes=C).float()
        if index is None:
            index = torch.arange(labels.size(0), device=labels.device)
        return self.ncod_loss(index, logits, y_onehot, phi_x_i, flag, epoch)

In [11]:
# Custom loss function for noisy labels
class NoisyLabelLoss(nn.Module):
    """Loss function that accounts for label noise"""
    def __init__(self, alpha=0.1, beta=0.5):
        super(NoisyLabelLoss, self).__init__()
        self.alpha = alpha  # Weight for confidence regularization
        self.beta = beta    # Weight for noise penalty
        self.ce_loss = nn.CrossEntropyLoss(reduction='none')

    def forward(self, outputs, labels):
        logits = outputs['logits']
        confidence = outputs['confidence'].squeeze()
        noise_score = outputs.get('noise_score', None)

        # Standard cross-entropy loss
        ce_loss = self.ce_loss(logits, labels) # Pass logits here

        # Weight loss by confidence (down-weight low-confidence samples)
        weighted_ce = ce_loss * confidence

        # Confidence regularization (encourage high confidence on clean samples)
        conf_reg = -torch.log(confidence + 1e-8)

        # Noise penalty
        noise_penalty = 0.0
        if noise_score is not None:
            noise_penalty = torch.mean(noise_score.squeeze())

        total_loss = (torch.mean(weighted_ce) +
                     self.alpha * torch.mean(conf_reg) +
                     self.beta * noise_penalty)

        return total_loss
        return total_loss

In [28]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.edge_attr, data.batch, data.y)
        loss = criterion(output, data.y, index=data.idx if hasattr(data, 'idx') else None, flag=0, epoch=current_epoch)
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        total_loss += loss.item()
        pred = output['logits'].argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [29]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data.x, data.edge_index, data.edge_attr, data.batch, data.y if calculate_accuracy else None)
            pred = output['logits'].argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output['logits'], data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [30]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [31]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [16]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [17]:
# Load the full dataset first to get labels and number of examples
full_dataset = GraphDataset("/kaggle/working/hackaton/datasets/A/train.json.gz", transform=add_zeros)

In [32]:
script_dir = os.getcwd()
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = 1
model = GINPlusModel(
        in_dim=8,
        edge_dim=8,
        hidden_dim=500,
        out_classes=6, # Ensure this matches the number of classes in your dataset
        num_layers=5,
        dropout=0.1,
        use_virtual_node=True,
        confidence_threshold=0.8
    ).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

# Get the labels and number of examples from the full dataset
all_labels = torch.tensor([data.y.item() for data in full_dataset])
num_examples = len(full_dataset)

# Instantiate ncodLoss with the required arguments
# Make sure the num_classes argument matches the actual number of classes
raw_ncod = ncodLoss(labels=all_labels, n=num_examples, C=6)
criterion = NcodLossWrapper(raw_ncod)
epochs=5

In [33]:
test_dir_name = os.path.basename(os.path.dirname("/kaggle/working/hackaton/datasets/A/test.json.gz"))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [34]:
if os.path.exists(checkpoint_path) and not os.path.basename(os.path.dirname("/kaggle/working/hackaton/datasets/A/test.json.gz")):
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [35]:
if os.path.basename(os.path.dirname("/kaggle/working/hackaton/datasets/A/test.json.gz")):
    
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

    num_epochs = epochs
    best_val_accuracy = 0.0   

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer, criterion, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Iterating training graphs:   0%|          | 0/1128 [00:00<?, ?batch/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2299x32 and 8x500)

In [ ]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

In [ ]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
    

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)